<a href="https://colab.research.google.com/github/mrhamedani/How-to-apply-popular-LLMs/blob/main/%E2%80%8C%E2%80%8CBBCNews_Chromadb_Liama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers --upgrade
!pip install sentence-transformers --upgrade
!pip install chromadb --upgrade


# Download and prepare the Dataset
 Log into Kaggle and go to the link https://www.kaggle.com/datasets/kotartemiy/topic-labeled-news-dataset to download

Using the Kaggle API JSON file, I saved the data set directly in the Google Colab temporary memory

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
import transformers
import sentence_transformers
import chromadb
from datetime import datetime
files.upload()

In [19]:
print("Transformers version:", transformers.__version__)
print("Sentence-Transformers version:", sentence_transformers.__version__)
print("ChromaDB version:", chromadb.__version__)

Transformers version: 4.48.1
Sentence-Transformers version: 3.4.0
ChromaDB version: 0.6.3


In [4]:
!kaggle datasets download -d gpreda/bbc-news
!unzip bbc-news.zip

Dataset URL: https://www.kaggle.com/datasets/gpreda/bbc-news
License(s): CC0-1.0
  0% 0.00/3.64M [00:00<?, ?B/s]
100% 3.64M/3.64M [00:00<00:00, 133MB/s]
Archive:  bbc-news.zip
  inflating: bbc_news.csv            


In [5]:
news = pd.read_csv('./bbc_news.csv')
MAX_NEWS = 1000
DOCUMENT="description"
TOPIC="title"

ChromaDB requires that the data has a unique identifier. You can achieve it with the statement below, which will create a new column called **Id**.

In [6]:
news["id"] = news.index
news.head(3)

,title,pubDate,guid,link,description,id
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...,0
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as...",1
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...,2


In [7]:
#Because it is just a example we select a small portion of News.
subset_news = news.head(MAX_NEWS)

In [28]:
import chromadb
from chromadb.config import Settings

chroma_client = chromadb.PersistentClient(path="./chromadb")

# Filling and Querying the ChromaDB Database
The Data in ChromaDB is stored in collections. If the collection previously exist is necessary to delete it.

In the next lines, the collection is created by calling the ***create_collection*** function in the ***chroma_client*** created above.

In [30]:

# ایجاد نام یکتا برای collection
collection_name = "news_collection" + datetime.now().strftime("%s")

# دریافت نام‌های همه collections
collection_names = chroma_client.list_collections()

# بررسی اینکه آیا نام collection مورد نظر وجود دارد
if collection_name in collection_names:
    chroma_client.delete_collection(name=collection_name)

# ساخت collection جدید
collection = chroma_client.create_collection(name=collection_name)

It's time to add the data to the collection. Using the function ***add*** you should inform, at least ***documents***, ***metadatas*** and ***ids***.
* In the **document** the full news text is stored, remember that it is contained in a different column for each Dataset.
* In **metadatas**, we can inform a list of topics.
* In **id** an unique identificator for each row must be informed. It MUST be unique! I'm creating the ID using the range of MAX_NEWS.

In [31]:
collection.add(
    documents=subset_news[DOCUMENT].tolist(),
    metadatas=[{TOPIC: topic} for topic in subset_news[TOPIC].tolist()],
    ids=[f"id{x}" for x in range(MAX_NEWS)],
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 39.4MiB/s]


In [32]:
results = collection.query(query_texts=["laptop"], n_results=10 )

print(results)

{'ids': [['id775', 'id707', 'id310', 'id587', 'id444', 'id751', 'id701', 'id862', 'id191', 'id740']], 'embeddings': None, 'documents': [['Photography student Thorsten Mjölnir captures the way students decorate their laptops.', 'Why sales of very basic mobile phones, without apps and internet connection, are increasing.', "What do you do when your collection of millions of books keeps growing but your bookshelves don't?", 'The developers of a powerful mini aircraft hope it will be used by the armed forces.', 'How tech is helping young families and couples regain their busy social lives after Covid.', 'Watch as Lee Zii Jia of Malaysia records a speed of 372km/h on his backhand point against Lakshya Sen of India in the All England Badminton Championships.', 'The Royal Mint has found a way to turn old circuit boards from phones, computers and TVs into gold.', 'A van was reportedly hijacked and driven to the venue, and a controlled explosion has since been carried out.', 'The Ukrainian pres

In [33]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

getado = collection.get(ids="id141",
                       include=["documents", "embeddings"])

In [34]:
word_vectors = getado["embeddings"]
word_list = getado["documents"]
word_vectors

array([[ 8.36053193e-02,  9.88176279e-03, -6.33521751e-02,
         6.83554960e-03,  9.83688012e-02,  6.62363842e-02,
         8.81884769e-02,  4.32013953e-03,  4.62584458e-02,
        -2.53718700e-02,  3.03161126e-02,  2.90436596e-02,
         3.73522788e-02,  1.74557697e-02, -3.08765993e-02,
        -4.47610579e-02, -7.60653382e-03, -5.80879115e-02,
        -4.53988910e-02, -6.97019175e-02, -3.50682274e-03,
         4.58132587e-02,  1.58201139e-02,  3.44685316e-02,
        -6.20314032e-02,  9.38326642e-02,  5.95623329e-02,
        -3.07523180e-02, -8.41464475e-02, -2.55898647e-02,
        -5.44909714e-03, -1.64661184e-02, -1.00670859e-01,
         5.22239953e-02, -6.52278960e-03,  6.44802079e-02,
         6.21989220e-02, -4.69207810e-03, -3.38835940e-02,
        -5.86173963e-04, -9.11896750e-02, -9.24299806e-02,
        -4.39610668e-02, -4.71336246e-02,  1.49744772e-03,
        -9.09678731e-03,  2.79689636e-02, -1.05015799e-01,
         6.03262410e-02,  5.91969416e-02, -2.59885769e-0

In [35]:
#!pip install -q einops==0.8.0

In [36]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
#model_id = "databricks/dolly-v2-3b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
lm_model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [37]:
pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    device_map="auto",
)

Device set to use cpu


In [38]:
question = "Can I buy a new Toshiba laptop?"
context = " ".join([f"#{str(i)}" for i in results["documents"][0]])
#context = context[0:5120]
prompt_template = f"""
Relevant context: {context}
Considering the relevant context, answer the question.
Question: {question}
Answer: """
prompt_template

"\nRelevant context: #Photography student Thorsten Mjölnir captures the way students decorate their laptops. #Why sales of very basic mobile phones, without apps and internet connection, are increasing. #What do you do when your collection of millions of books keeps growing but your bookshelves don't? #The developers of a powerful mini aircraft hope it will be used by the armed forces. #How tech is helping young families and couples regain their busy social lives after Covid. #Watch as Lee Zii Jia of Malaysia records a speed of 372km/h on his backhand point against Lakshya Sen of India in the All England Badminton Championships. #The Royal Mint has found a way to turn old circuit boards from phones, computers and TVs into gold. #A van was reportedly hijacked and driven to the venue, and a controlled explosion has since been carried out. #The Ukrainian president reveals his location in Kyiv in a new video shared on social media. #The Royal Mint has found a way to turn old circuit boards

In [39]:
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])


Relevant context: #Photography student Thorsten Mjölnir captures the way students decorate their laptops. #Why sales of very basic mobile phones, without apps and internet connection, are increasing. #What do you do when your collection of millions of books keeps growing but your bookshelves don't? #The developers of a powerful mini aircraft hope it will be used by the armed forces. #How tech is helping young families and couples regain their busy social lives after Covid. #Watch as Lee Zii Jia of Malaysia records a speed of 372km/h on his backhand point against Lakshya Sen of India in the All England Badminton Championships. #The Royal Mint has found a way to turn old circuit boards from phones, computers and TVs into gold. #A van was reportedly hijacked and driven to the venue, and a controlled explosion has since been carried out. #The Ukrainian president reveals his location in Kyiv in a new video shared on social media. #The Royal Mint has found a way to turn old circuit boards f

In [40]:
import chromadb
chroma_client_2 = chromadb.PersistentClient(path="./chromadb")

In [41]:
collection2 = chroma_client_2.get_collection(name=collection_name)
results2 = collection.query(query_texts=["laptop"], n_results=10 )


In [42]:
print(results2)

{'ids': [['id775', 'id707', 'id310', 'id587', 'id444', 'id751', 'id701', 'id862', 'id191', 'id740']], 'embeddings': None, 'documents': [['Photography student Thorsten Mjölnir captures the way students decorate their laptops.', 'Why sales of very basic mobile phones, without apps and internet connection, are increasing.', "What do you do when your collection of millions of books keeps growing but your bookshelves don't?", 'The developers of a powerful mini aircraft hope it will be used by the armed forces.', 'How tech is helping young families and couples regain their busy social lives after Covid.', 'Watch as Lee Zii Jia of Malaysia records a speed of 372km/h on his backhand point against Lakshya Sen of India in the All England Badminton Championships.', 'The Royal Mint has found a way to turn old circuit boards from phones, computers and TVs into gold.', 'A van was reportedly hijacked and driven to the venue, and a controlled explosion has since been carried out.', 'The Ukrainian pres